# Convolutional Neural Networks

From now on, it might be better to run these Jupyter Notebooks on Google Colab. From the Colab toolbar, select *Runtime* > *Change runtime type* > *T4 GPU* > *Save* before running the Notebook.

In [ ]:
from keras import Model
from keras import Input
from keras.layers import Dense
from keras.layers import Rescaling
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

from keras.optimizers import RMSprop

from keras.datasets.mnist import load_data

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.datasets import mnist

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

## Reshaping the MNIST Dataset

This is what we did in the previous lecture:

In [ ]:
# X_train = X_train.reshape((60000, 28 * 28))
# X_test = X_test.reshape((10000, 28 * 28))

But, from now on, we don't flatten. Instead, we add a dimension so that we have a rank 3 tensor:

In [ ]:
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))

## A 2D ConvNet for MNIST

In [ ]:
inputs = Input(shape=(28, 28, 1))
x = Rescaling(scale=1./255)(inputs)
x = Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation="relu")(x)
x = Flatten()(x)
x = Dense(units=64, activation="relu")(x)
outputs = Dense(units=10, activation="softmax")(x)
convnet = Model(inputs, outputs)

In [ ]:
convnet.compile(optimizer=RMSprop(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
convnet.summary()

Do you understand all the numbers in the table above?

In [ ]:
convnet.fit(X_train, y_train, epochs=20, batch_size=32, 
            verbose=0, validation_split=0.25, 
            callbacks=[EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)])

We'll explain early stopping in a subsequent lecture, when we discuss overfitting. 

In [ ]:
test_loss, test_acc = convnet.evaluate(X_test, y_test)
test_acc

A tiny bit more accurate than in the previous Notebook.